In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import sklearn
import scipy as sp
import statsmodels as sm
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision as tv
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.utils as vutils
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss

## Data preparation

In [129]:
# Data are sorted in ./task1
Train_DataX = pd.read_csv("./task1/X_train.csv", index_col=[0], header=[0, 1, 2])
Valid_DataX = pd.read_csv("./task1/X_valid.csv", index_col=[0], header=[0, 1, 2])
X_Test = pd.read_csv('./task1/X_test.csv', header=[0,1,2], index_col=[0])

In [130]:
Train_DataX.head()

LEVEL2               alanine aminotransferase                                \
Aggregation Function                     mask                                 
hours_in                                    0    1    2    3    4    5    6   
3_145834_211552                           1.0  0.0  0.0  0.0  0.0  0.0  0.0   
6_107064_228232                           1.0  0.0  0.0  0.0  0.0  0.0  0.0   
9_150750_220597                           0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11_194540_229441                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
12_112213_232669                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   

LEVEL2                               ... white blood cell count urine         \
Aggregation Function                 ...          time_since_measured          
hours_in                7    8    9  ...                           14     15   
3_145834_211552       0.0  0.0  0.0  ...                          6.0    7.0   
6_107064_228232       0.0  0.0  0.0  ...                         30.0   31.0   
9_150750_220597       0.0  0.0  0.0  ...                         54.0   55.0   
11_194540_229441      0.0  0.0  0.0  ...                         78.0   79.0   
12_112213_232669      0.0  0.0  0.0  ...                        102.0  103.0   

LEVEL2                                                                        
Aggregation Function                                                          
hours_in                 16     17     18     19     20     21     22     23  
3_145834_211552         8.0    9.0   10.0   11.0   12.0   13.0   14.0   15.0  
6_107064_228232        32.0   33.0   34.0   35.0   36.0   37.0   38.0   39.0  
9_150750_220597        56.0   57.0   58.0   59.0   60.0   61.0   62.0   63.0  
11_194540_229441       80.0   81.0   82.0   83.0   84.0   85.0   86.0   87.0  
12_112213_232669      104.0  105.0  106.0  107.0  108.0  109.0  110.0  111.0  

[5 rows x 7488 columns]

In [131]:
# For each header[0], and header[1] == 'mean', for each header[2], get the difference between the max and min of all its layers
diff_train = Train_DataX[[col for col in Train_DataX.columns if col[1] == 'mean']].groupby(level=0, axis=1).max() - Train_DataX[[col for col in Train_DataX.columns if col[1] == 'mean']].groupby(level=0, axis=1).min()
diff_valid = Valid_DataX[[col for col in Valid_DataX.columns if col[1] == 'mean']].groupby(level=0, axis=1).max() - Valid_DataX[[col for col in Valid_DataX.columns if col[1] == 'mean']].groupby(level=0, axis=1).min()
diff_test = X_Test[[col for col in X_Test.columns if col[1] == 'mean']].groupby(level=0, axis=1).max() - X_Test[[col for col in X_Test.columns if col[1] == 'mean']].groupby(level=0, axis=1).min()

# change above header extend to header[0] == header, header[1] == 'mean', header[2] == 'diff'
diff_train.columns = pd.MultiIndex.from_tuples([(col, 'mean', 'diff') for col in diff_train.columns])
diff_valid.columns = pd.MultiIndex.from_tuples([(col, 'mean', 'diff') for col in diff_valid.columns])
diff_test.columns = pd.MultiIndex.from_tuples([(col, 'mean', 'diff') for col in diff_test.columns])

In [132]:
# For each header[0], and header[1] == 'time_since_measured', get the maximum value of header[2] and store as a new frame
max_time_train = Train_DataX[[col for col in Train_DataX.columns if col[1] == 'time_since_measured']].groupby(level=0, axis=1).max()
max_time_valid = Valid_DataX[[col for col in Valid_DataX.columns if col[1] == 'time_since_measured']].groupby(level=0, axis=1).max()
max_time_test = X_Test[[col for col in X_Test.columns if col[1] == 'time_since_measured']].groupby(level=0, axis=1).max()

# For each header[0], and header[1] == 'time_since_measured', get the minimum value of header[2] and store as a new frame
min_time_train = Train_DataX[[col for col in Train_DataX.columns if col[1] == 'time_since_measured']].groupby(level=0, axis=1).min()
min_time_valid = Valid_DataX[[col for col in Valid_DataX.columns if col[1] == 'time_since_measured']].groupby(level=0, axis=1).min()
min_time_test = X_Test[[col for col in X_Test.columns if col[1] == 'time_since_measured']].groupby(level=0, axis=1).min()

# Get the difference between the max and min of all the time_since_measured layers
diff_time_train = max_time_train - min_time_train
diff_time_valid = max_time_valid - min_time_valid
diff_time_test = max_time_test - min_time_test

# change above header extend to header[0] == header, header[1] == 'time_since_measured', header[2] == 'diff'
diff_time_train.columns = pd.MultiIndex.from_tuples([(col, 'time_since_measured', 'diff') for col in diff_time_train.columns])
diff_time_valid.columns = pd.MultiIndex.from_tuples([(col, 'time_since_measured', 'diff') for col in diff_time_valid.columns])
diff_time_test.columns = pd.MultiIndex.from_tuples([(col, 'time_since_measured', 'diff') for col in diff_time_test.columns])

In [133]:
# Drop all the time_since_measured layers
Train_DataX = Train_DataX[[col for col in Train_DataX.columns if col[1] != 'time_since_measured']]
Valid_DataX = Valid_DataX[[col for col in Valid_DataX.columns if col[1] != 'time_since_measured']]
X_Test = X_Test[[col for col in X_Test.columns if col[1] != 'time_since_measured']]

In [134]:
# for all column with header [1] == 'mask' and header[1] == 'mean', if their header[0] are same and header [2] are same, value of 'mean' = value of 'mean' * value of 'mask'
for col in Train_DataX.columns:
    if col[1] == 'mean':
        Train_DataX[col] = Train_DataX[col] * Train_DataX[(col[0], 'mask', col[2])]
for col in Valid_DataX.columns:
    if col[1] == 'mean':
        Valid_DataX[col] = Valid_DataX[col] * Valid_DataX[(col[0], 'mask', col[2])]
for col in X_Test.columns:
    if col[1] == 'mean':
        X_Test[col] = X_Test[col] * X_Test[(col[0], 'mask', col[2])]

In [135]:
# 1. Let header[1] == 'mask' be sum of all its header[2] layers
mask_train = Train_DataX[[col for col in Train_DataX.columns if col[1] == 'mask']].groupby(level=0, axis=1).sum()
mask_valid = Valid_DataX[[col for col in Valid_DataX.columns if col[1] == 'mask']].groupby(level=0, axis=1).sum()
mask_test = X_Test[[col for col in X_Test.columns if col[1] == 'mask']].groupby(level=0, axis=1).sum()

# 2. For each header[0], and header[1] == 'mean', regardless of header[2], get the mean of all its layers
mean_train = Train_DataX[[col for col in Train_DataX.columns if col[1] == 'mean']].groupby(level=0, axis=1).mean()
mean_valid = Valid_DataX[[col for col in Valid_DataX.columns if col[1] == 'mean']].groupby(level=0, axis=1).mean()
mean_test = X_Test[[col for col in X_Test.columns if col[1] == 'mean']].groupby(level=0, axis=1).mean()


# 3. For all header in mean_train, mean_valid, mean_test, its value = value * 24 / value of mask 
for col in mean_train.columns:
    mean_train[col] = mean_train[col] * 24 / mask_train[col]
for col in mean_valid.columns:
    mean_valid[col] = mean_valid[col] * 24 / mask_valid[col]
for col in mean_test.columns:
    mean_test[col] = mean_test[col] * 24 / mask_test[col]

# Change the header of mask_train, mask_valid, mask_test to header[0] == header, header[1] == 'mask', header[2] == 'sum'
mask_train.columns = pd.MultiIndex.from_tuples([(col, 'mask', 'sum') for col in mask_train.columns])
mask_valid.columns = pd.MultiIndex.from_tuples([(col, 'mask', 'sum') for col in mask_valid.columns])
mask_test.columns = pd.MultiIndex.from_tuples([(col, 'mask', 'sum') for col in mask_test.columns])

# Change the header of mean_train, mean_valid, mean_test to header[0] == header, header[1] == 'mean', header[2] == 'avg'
mean_train.columns = pd.MultiIndex.from_tuples([(col, 'mean', 'avg') for col in mean_train.columns])
mean_valid.columns = pd.MultiIndex.from_tuples([(col, 'mean', 'avg') for col in mean_valid.columns])
mean_test.columns = pd.MultiIndex.from_tuples([(col, 'mean', 'avg') for col in mean_test.columns])

In [136]:
mean_train.head()

,alanine aminotransferase,albumin,albumin ascites,albumin pleural,albumin urine,alkaline phosphate,anion gap,asparate aminotransferase,basophils,bicarbonate,...,tidal volume set,tidal volume spontaneous,total protein,total protein urine,troponin-i,troponin-t,venous pvo2,weight,white blood cell count,white blood cell count urine
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
,avg,avg,avg,avg,avg,avg,avg,avg,avg,avg,...,avg,avg,avg,avg,avg,avg,avg,avg,avg,avg
3_145834_211552,-0.254460,-1.979855,NaN,NaN,NaN,-0.318615,0.931458,-0.226618,NaN,-1.694872,...,0.440628,NaN,NaN,NaN,NaN,NaN,NaN,1.026004,0.634186,0.160067
6_107064_228232,-0.256599,-0.251806,NaN,NaN,NaN,-0.422405,1.723627,-0.254291,-0.742403,-1.444821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.181589,NaN
9_150750_220597,-0.269432,NaN,NaN,NaN,NaN,-0.367050,-0.330641,-0.271686,2.097036,1.019964,...,1.039571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.181589,NaN
11_194540_229441,NaN,NaN,NaN,NaN,NaN,NaN,-0.088963,NaN,NaN,0.484141,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.139623,NaN
12_112213_232669,NaN,NaN,NaN,NaN,NaN,NaN,1.240269,NaN,NaN,-1.694872,...,0.680205,-1.333059,NaN,NaN,NaN,-0.317391,NaN,NaN,-0.414731,NaN


In [137]:
mask_train.head()

,alanine aminotransferase,albumin,albumin ascites,albumin pleural,albumin urine,alkaline phosphate,anion gap,asparate aminotransferase,basophils,bicarbonate,...,tidal volume set,tidal volume spontaneous,total protein,total protein urine,troponin-i,troponin-t,venous pvo2,weight,white blood cell count,white blood cell count urine
,mask,mask,mask,mask,mask,mask,mask,mask,mask,mask,...,mask,mask,mask,mask,mask,mask,mask,mask,mask,mask
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
3_145834_211552,1.0,1.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,4.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0
6_107064_228232,1.0,1.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9_150750_220597,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,2.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
11_194540_229441,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
12_112213_232669,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,...,7.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0


In [145]:
# Concatenate all the frames together which is original data, mask, mean, diff, diff_time
train_data = pd.read_csv("./task1/X_train.csv", index_col=[0], header=[0, 1, 2])
train_data = pd.concat([train_data, mask_train, mean_train, diff_train, diff_time_train], axis=1)
valid_data = pd.read_csv("./task1/X_valid.csv", index_col=[0], header=[0, 1, 2])
valid_data = pd.concat([valid_data, mask_valid, mean_valid, diff_valid, diff_time_valid], axis=1)
test_data = pd.read_csv("./task1/X_test.csv", index_col=[0], header=[0, 1, 2])
test_data = pd.concat([test_data, mask_test, mean_test, diff_test, diff_time_test], axis=1)

In [146]:
# Reindex the columns to make sure the order is correct
newcolums = []
for col in train_data.columns:
    if col[2] == 'diff' or col[2] == 'avg' or col[2] == 'sum':
        continue
    newcolums.append(col)
    if col[2] == '23':
        if col[1] == 'mask':
            newcolums.append((col[0], 'mask', 'sum'))
        elif col[1] == 'mean':
            newcolums.append((col[0], 'mean', 'avg'))
            newcolums.append((col[0], 'mean', 'diff'))
        elif col[1] == 'time_since_measured':
            newcolums.append((col[0], 'time_since_measured', 'diff'))
train_data = train_data[newcolums]
valid_data = valid_data[newcolums]
test_data = test_data[newcolums]

In [147]:
# Store the prepared data in ./new_data2
train_data.to_csv('./SuperData/X_train.csv')
valid_data.to_csv('./SuperData/X_valid.csv')
test_data.to_csv('./SuperData/X_test.csv')

In [148]:
train_data

LEVEL2               alanine aminotransferase                                \
Aggregation Function                     mask                                 
hours_in                                    0    1    2    3    4    5    6   
3_145834_211552                           1.0  0.0  0.0  0.0  0.0  0.0  0.0   
6_107064_228232                           1.0  0.0  0.0  0.0  0.0  0.0  0.0   
9_150750_220597                           0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11_194540_229441                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
12_112213_232669                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                                       ...  ...  ...  ...  ...  ...  ...   
99966_167228_252173                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99973_150202_275083                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99982_151454_221194                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99991_151118_226241                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99995_137810_229633                       0.0  1.0  1.0  0.0  0.0  0.0  0.0   

LEVEL2                               ... white blood cell count urine         \
Aggregation Function                 ...          time_since_measured          
hours_in                7    8    9  ...                           15     16   
3_145834_211552       0.0  0.0  0.0  ...                          7.0    8.0   
6_107064_228232       0.0  0.0  0.0  ...                         31.0   32.0   
9_150750_220597       0.0  0.0  0.0  ...                         55.0   56.0   
11_194540_229441      0.0  0.0  0.0  ...                         79.0   80.0   
12_112213_232669      0.0  0.0  0.0  ...                        103.0  104.0   
...                   ...  ...  ...  ...                          ...    ...   
99966_167228_252173   0.0  0.0  0.0  ...                         87.0   88.0   
99973_150202_275083   0.0  0.0  0.0  ...                        111.0  112.0   
99982_151454_221194   0.0  0.0  0.0  ...                        135.0  136.0   
99991_151118_226241   0.0  0.0  0.0  ...                         15.0   16.0   
99995_137810_229633   0.0  0.0  0.0  ...                         39.0   40.0   

LEVEL2                                                                        
Aggregation Function                                                          
hours_in                 17     18     19     20     21     22     23   diff  
3_145834_211552         9.0   10.0   11.0   12.0   13.0   14.0   15.0  100.0  
6_107064_228232        33.0   34.0   35.0   36.0   37.0   38.0   39.0   23.0  
9_150750_220597        57.0   58.0   59.0   60.0   61.0   62.0   63.0   23.0  
11_194540_229441       81.0   82.0   83.0   84.0   85.0   86.0   87.0   23.0  
12_112213_232669      105.0  106.0  107.0  108.0  109.0  110.0  111.0   23.0  
...                     ...    ...    ...    ...    ...    ...    ...    ...  
99966_167228_252173    89.0   90.0   91.0   92.0   93.0   94.0   95.0   23.0  
99973_150202_275083   113.0  114.0  115.0  116.0  117.0  118.0  119.0   23.0  
99982_151454_221194   137.0  138.0  139.0  140.0  141.0  142.0  143.0   23.0  
99991_151118_226241    17.0   18.0   19.0   20.0   21.0   22.0   23.0   23.0  
99995_137810_229633    41.0   42.0   43.0   44.0   45.0   46.0   47.0   23.0  

[16760 rows x 7904 columns]